In [124]:
import pandas as pd
import warnings
import numpy as np
# Игнорируем предупреждения
warnings.filterwarnings('ignore')

In [125]:
# Читаем excel документ с помощью Pandas 
if __name__ == "__main__":
    data = pd.read_excel("srez1.xlsx", sheet_name="ТИ")

In [126]:
# Извлекаем из строки данные в скобках и последнее слово
def convert(x: str) -> str:
    ans = ""
    p = 0
    for i in range(len(x)):
        if x[i] == '(':
            p += 1
            if(p == 1):
                ans += "("
            j = i + 1
            while x[j] != ')' and j < len(x):
                ans += x[j]
                j += 1
            if(p == 1):
                ans += ")"
            ans += "_"
    i = len(x) - 1
    tmp = ""
    while(x[i] != ' '):
        tmp += x[i]
        i -= 1
    ans += tmp[::-1]
    return ans         
# В случае если нам понадобиться убрать лишний столбец
# data.drop(labels = 'Unnamed: 5', axis = 1, inplace = True)

In [127]:
%%time
# Пробегаемся по интересующему нас столбцу и конвертируем строки
for i in range(len(data['NAME_MEAS'])):
    s = data['NAME_MEAS'][i]
    ss = convert(s)
    # записываем получившиеся данные в свободный столбец
    data['PF_NAME_MEAS'][i] = ss

CPU times: total: 141 ms
Wall time: 149 ms


In [128]:
data

,NAME_MEAS,IECaddr,MEAS_VAL,Counter,PF_NAME_MEAS
0,ТИ: Жылга (839) Узел U,11353,!0,4,(839)_U
1,ТИ: Жылга (839) Узел Pн,11354,*0,0,(839)_Pн
2,ТИ: Жылга (839) Узел Qн,11355,*0,0,(839)_Qн
3,ТИ: Ташкент (2925) Л-2429 (839-2925-3) Pij,11438,15.24,174,(2925)_839-2925-3_Pij
4,ТИ: Ташкент (2925) Л-2429 (839-2925-3) Qij,11439,!0,126,(2925)_839-2925-3_Qij
...,...,...,...,...,...
2221,КУРЧАТОВСКАЯ (10002) Pг,13156,859.21,146,(10002)_Pг
2222,КУРЧАТОВСКАЯ (10002) Qг,13157,280.22,244,(10002)_Qг
2223,ЮЖНАЯ (4701) U,13158,505.23,49,(4701)_U
2224,ЮЖНАЯ (4701) Pн,13159,1167.8,154,(4701)_Pн


In [129]:
# Если нужно сверять со вторым листом того же файла или другого, то используйте ниже
df = pd.read_excel("srez1.xlsx", sheet_name="PFmeas")
                            
# OR
# Нужно было создать так как не было данных второго листа
# Записываем данные в отдельный DF для проверки
#df=data[['Unnamed: 5', 'Unnamed: 2', 'Unnamed: 3']]

In [130]:
# Для проверки
# Перезаписываем хотя бы одну строку в столбце, для проверки далее
# data['NAME_MEAS'][0] = '(839)_Pн'

In [131]:
df

,PF_NAME_MEAS,MEAS_VAL,OutOfServ
0,(839)_U,NaN,NaN
1,(839)_Pн,NaN,NaN
2,(839)_Qн,NaN,NaN
3,(2925)_839-2925-3_Pij,NaN,NaN
4,(2925)_839-2925-3_Qij,NaN,NaN
5,(839)_Рг,NaN,NaN


In [132]:
# Если данные первого столбца второго листа не совпадают с данными последнего столбца первого листа, то 1, 
# если совпадают то второй столбец заполняется измерениям, а третий столбец 0 
for i in range(len(df['PF_NAME_MEAS'])): 
    for j in range(len(data['PF_NAME_MEAS'])):
        if (df['PF_NAME_MEAS'][i] == data['PF_NAME_MEAS'][j]): 
            df['MEAS_VAL'][i] = data['MEAS_VAL'][j] 
            df['OutOfServ'][i] = '0' 
            break
        else: 
            df['MEAS_VAL'][i] = '0' 
            df['OutOfServ'][i] = '1' 
                        
df


,PF_NAME_MEAS,MEAS_VAL,OutOfServ
0,(839)_U,!0,0
1,(839)_Pн,*0,0
2,(839)_Qн,*0,0
3,(2925)_839-2925-3_Pij,15.24,0
4,(2925)_839-2925-3_Qij,!0,0
5,(839)_Рг,0,1


In [133]:
# Сохраняем данные в новом excel документе
from pandas.io.excel import ExcelWriter
# sheet_name - листы в excel
# Можно указать тот же excel файл или просто указав другой name создать новый
with ExcelWriter('srez1_res.xlsx') as writer:
    data.to_excel(writer, sheet_name="ТИ", index=False) # Сохраняем первый лист с конвертированными данными
    df.to_excel(writer, sheet_name="PFmeas", index=False) # Сохраняем второй лист с проверочными данными

In [134]:
# Проверяем наш записанный excel файл и сортируем по первому столбцу
new_data = pd.read_excel("srez1_res.xlsx", sheet_name="PFmeas")
new_data